In [6]:
"""
TinyLM: A Tiny Language Model from Scratch
===============================================
- Code data: bigcode/the-stack-smol-python
- NL data : HuggingFaceTB/smollm-corpus
- 8-layer decoder-only transformer
- Causal LM objective (next-token prediction)
"""

import math
import random
import time
from dataclasses import dataclass
from itertools import cycle

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset
from transformers import GPT2TokenizerFast
from tqdm import tqdm


In [7]:
# ============================================================
# 1. CONFIG
# ============================================================

@dataclass
class Config:
    # model
    vocab_size: int = 50257
    d_model: int = 512
    n_heads: int = 8
    n_layers: int = 8
    d_ff: int = 2048
    block_size: int = 256

    # training
    batch_size: int = 8
    lr_max: float = 3e-4
    lr_min: float = 1e-5
    warmup_steps: int = 1_000
    max_steps: int = 80_000
    log_interval: int = 100
    eval_interval: int = 2_000
    weight_decay: float = 0.1

    # data mix
    p_code: float = 0.8

    # runtime
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    dtype: torch.dtype = (
        torch.bfloat16
        if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
        else torch.float16
    )


cfg = Config()
print(cfg)

Config(vocab_size=50257, d_model=512, n_heads=8, n_layers=8, d_ff=2048, block_size=256, batch_size=8, lr_max=0.0003, lr_min=1e-05, warmup_steps=1000, max_steps=80000, log_interval=100, eval_interval=2000, weight_decay=0.1, p_code=0.8, device='cuda', dtype=torch.bfloat16)


In [8]:
# ============================================================
# 2. LEARNING RATE SCHEDULER
# ============================================================

def get_lr(step: int) -> float:
    """Cosine decay with warmup."""
    if step < cfg.warmup_steps:
        return cfg.lr_max * step / cfg.warmup_steps

    progress = (step - cfg.warmup_steps) / max(1, (cfg.max_steps - cfg.warmup_steps))
    cosine = 0.5 * (1 + math.cos(math.pi * progress))
    return cfg.lr_min + (cfg.lr_max - cfg.lr_min) * cosine

In [9]:
# ============================================================
# 3. TOKENIZER (GPT-NeoX SentencePiece tokenizer)
# ============================================================

from transformers import AutoTokenizer

# Use GPT-NeoX tokenizer — supports all UTF-8 (best for code + NL)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

# Ensure a pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

pad_token_id = tokenizer.pad_token_id

# Update vocab size in config
cfg.vocab_size = tokenizer.vocab_size

print("Tokenizer vocab size:", cfg.vocab_size)
print("Pad token ID:", pad_token_id)


Tokenizer vocab size: 50254
Pad token ID: 0


In [10]:
# ============================================================
# 4. DATA
# ============================================================

print("Loading datasets...")

ds_code = load_dataset(
    "bigcode/the-stack-smol",
    data_dir="data/python",
    split="train",
    streaming=True
    )

ds_nl = load_dataset(
    "HuggingFaceTB/smollm-corpus",
    "cosmopedia-v2",
    split="train",
    streaming=True
)

print("Buffering datasets into memory... (this may take ~11000–30s)")

MAX_BUF = 50_000   # adjust accordig to RAM; 50k is safe

code_buf = [row["content"] for row in ds_code.take(MAX_BUF)]
nl_buf   = [row["text"]     for row in ds_nl.take(MAX_BUF)]

print(f"Buffered {len(code_buf)} code samples and {len(nl_buf)} NL samples.")

# Calculate data statistics
def count_tokens(texts):
    total = 0
    for text in texts:
        ids = tokenizer(text, truncation=True, max_length=cfg.block_size, return_tensors="pt").input_ids
        total += ids.numel()
    return total

code_tokens = count_tokens(code_buf)
nl_tokens = count_tokens(nl_buf)
total_tokens = code_tokens + nl_tokens

print(f"\n📊 Data Statistics:")
print(f"  Code tokens: {code_tokens:,}")
print(f"  NL tokens: {nl_tokens:,}")
print(f"  Total tokens: {total_tokens:,}")
print(f"  Total samples: {len(code_buf) + len(nl_buf):,}")

# Estimate epochs
tokens_per_step = cfg.batch_size * (cfg.block_size - 1)
total_tokens_training = cfg.max_steps * tokens_per_step
num_epochs = total_tokens_training / total_tokens
print(f"  Tokens per step: {tokens_per_step:,}")
print(f"  Total training tokens: {total_tokens_training:,}")
print(f"  Estimated epochs: {num_epochs:.2f}")

def encode(text):
      ids = tokenizer(
          text,
          truncation=True,
          max_length=cfg.block_size,
          padding="max_length",
          return_tensors="pt",
      ).input_ids.squeeze(0)
      # Clamp token IDs to valid range
      ids = torch.clamp(ids, 0, cfg.vocab_size - 1)
      return ids

def data_stream():
    while True:
        if random.random() < cfg.p_code:
            text = random.choice(code_buf)
        else:
            text = random.choice(nl_buf)
        ids = encode(text)
        x = ids[:-1]
        y = ids[1:]
        yield x, y

train_iter = cycle(data_stream())


Loading datasets...


Loading datasets...


Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading datasets...


Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading datasets...


Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Buffering datasets into memory... (this may take ~10–30s)
Buffered 10000 code samples and 100000 NL samples.
Buffered 10000 code samples and 100000 NL samples.

📊 Data Statistics:
  Code tokens: 2,291,279
  NL tokens: 25,504,950
  Total tokens: 27,796,229
  Total samples: 110,000
  Tokens per step: 2,040
  Total training tokens: 163,200,000
  Estimated epochs: 5.87

📊 Data Statistics:
  Code tokens: 2,291,279
  NL tokens: 25,504,950
  Total tokens: 27,796,229
  Total samples: 110,000
  Tokens per step: 2,040
  Total training tokens: 163,200,000
  Estimated epochs: 5.87


In [11]:
# ============================================================
# 5. MODEL
# ============================================================

class CausalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, block_size):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.proj = nn.Linear(d_model, d_model)

        mask = torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, C = x.shape

        qkv = self.qkv(x)
        q, k, v = qkv.split(C, dim=2)

        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)

        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(y)


class Block(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, block_size):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = CausalSelfAttention(d_model, n_heads, block_size)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x


class TinyDecoderLM(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg

        self.tok_emb = nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_emb = nn.Embedding(cfg.block_size, cfg.d_model)

        self.blocks = nn.ModuleList([
            Block(cfg.d_model, cfg.n_heads, cfg.d_ff, cfg.block_size)
            for _ in range(cfg.n_layers)
        ])

        self.ln_f = nn.LayerNorm(cfg.d_model)
        self.head = nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Linear, nn.Embedding)):
            nn.init.normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.zeros_(m.bias)

    def forward(self, idx):
        B, T = idx.shape
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)

        x = self.tok_emb(idx) + self.pos_emb(pos)
        for blk in self.blocks:
            x = blk(x)
        x = self.ln_f(x)
        return self.head(x)


model = TinyDecoderLM(cfg).to(cfg.device, memory_format=torch.contiguous_format)
model = model.to(dtype=cfg.dtype)
print(f"Model params: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

Model params: 76.81M


In [12]:
# ============================================================
# 6. OPTIMIZER
# ============================================================

def create_optimizer(model):
    decay, no_decay = [], []
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if p.ndim >= 2 and "bias" not in name:
            decay.append(p)
        else:
            no_decay.append(p)

    groups = [
        {"params": decay, "weight_decay": cfg.weight_decay},
        {"params": no_decay, "weight_decay": 0.0},
    ]

    fused = ("fused" in torch.optim.AdamW.__init__.__code__.co_varnames)

    return torch.optim.AdamW(
        groups,
        lr=cfg.lr_max,
        betas=(0.9, 0.95),
        eps=1e-8,
        fused=fused,
    )

optimizer = create_optimizer(model)

In [13]:
# ============================================================
# 7. TRAINING LOOP
# ============================================================

def get_batch():
    xs = []
    ys = []
    for _ in range(cfg.batch_size):
        x, y = next(train_iter)        # x, y: (T,)
        xs.append(x.unsqueeze(0))      # (1, T)
        ys.append(y.unsqueeze(0))
    x = torch.cat(xs, dim=0)          # (B, T)
    y = torch.cat(ys, dim=0)          # (B, T)
    return x, y


def debug_check_batch(x, y, step, context="train"):
    """
    CPU-side sanity checks to catch issues that cause CUDA device-side asserts:
    - target indices out of range
    - wrong shapes
    Run only for first few steps to keep overhead tiny.
    """
    x_cpu = x.detach().cpu()
    y_cpu = y.detach().cpu()

    if x_cpu.ndim != 2 or y_cpu.ndim != 2:
        raise ValueError(
            f"[{context}] step {step}: expected (B, T) tensors, "
            f"got x.shape={tuple(x_cpu.shape)}, y.shape={tuple(y_cpu.shape)}"
        )

    vmax_x = int(x_cpu.max().item())
    vmin_x = int(x_cpu.min().item())
    vmax_y = int(y_cpu.max().item())
    vmin_y = int(y_cpu.min().item())

    if vmin_x < 0 or vmin_y < 0 or vmax_x >= cfg.vocab_size or vmax_y >= cfg.vocab_size:
        raise ValueError(
            f"[{context}] step {step}: token IDs out of range!\n"
            f"  x.min={vmin_x}, x.max={vmax_x}, "
            f"  y.min={vmin_y}, y.max={vmax_y}, "
            f"  cfg.vocab_size={cfg.vocab_size}"
        )

    # extra: check logits last dim matches vocab
    # (can't check here, but we'll assert after forward)


scaler = torch.cuda.amp.GradScaler(enabled=(cfg.dtype == torch.float16))

print("\n" + "="*80)
print("🚀 STARTING TRAINING")
print("="*80)

t0 = time.time()
running_loss = 0.0

# Calculate epoch info
tokens_per_step = cfg.batch_size * (cfg.block_size - 1)

pbar = tqdm(range(1, cfg.max_steps + 1), desc="Training", unit="step", 
            ncols=120, colour="green", position=0, leave=True)

for step in pbar:
    
    # Calculate current epoch
    current_epoch = (step * tokens_per_step) / total_tokens

    # --------------------------------------------------------
    # LR schedule
    # --------------------------------------------------------
    lr = get_lr(step)
    for pg in optimizer.param_groups:
        pg["lr"] = lr

    # --------------------------------------------------------
    # batch
    # --------------------------------------------------------
    x, y = get_batch()            # (B, T), integer token IDs
    x = x.to(cfg.device)
    y = y.to(cfg.device)

    # run sanity checks on first few steps (CPU-side)
    if step <= 5:
        debug_check_batch(x, y, step, context="train")

    # --------------------------------------------------------
    # forward pass
    # --------------------------------------------------------
    with torch.cuda.amp.autocast(enabled=(cfg.dtype == torch.float16)):
        logits = model(x)         # (B, T, vocab_size)
        if step <= 5:
            # shape check for safety
            if logits.ndim != 3 or logits.size(-1) != cfg.vocab_size:
                raise ValueError(
                    f"[train] step {step}: logits shape invalid. "
                    f"Expected (B, T, {cfg.vocab_size}), got {tuple(logits.shape)}"
                )

        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            y.view(-1),
            ignore_index=pad_token_id,
        )

    # --------------------------------------------------------
    # backward
    # --------------------------------------------------------
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)

    # --------------------------------------------------------
    # logging
    # --------------------------------------------------------
    running_loss += loss.item()
    if step % cfg.log_interval == 0:
        avg = running_loss / cfg.log_interval
        ppl = math.exp(avg) if avg < 20 else float("inf")
        elapsed = time.time() - t0
        pbar.set_postfix({
            "epoch": f"{current_epoch:.2f}", 
            "loss": f"{avg:.4f}", 
            "ppl": f"{ppl:.2f}", 
            "lr": f"{lr:.2e}"
        })
        running_loss = 0.0
        t0 = time.time()

    # --------------------------------------------------------
    # eval + checkpoint
    # --------------------------------------------------------
    if step % cfg.eval_interval == 0 or step == cfg.max_steps:
        model.eval()
        eval_losses = []
        with torch.no_grad():
            for _ in range(32):
                x_eval, y_eval = get_batch()   # 🔁 use get_batch here too
                x_eval = x_eval.to(cfg.device)
                y_eval = y_eval.to(cfg.device)

                if step <= 5:
                    debug_check_batch(x_eval, y_eval, step, context="eval")

                logits_eval = model(x_eval)
                if step <= 5:
                    if logits_eval.ndim != 3 or logits_eval.size(-1) != cfg.vocab_size:
                        raise ValueError(
                            f"[eval] step {step}: logits shape invalid. "
                            f"Expected (B, T, {cfg.vocab_size}), got {tuple(logits_eval.shape)}"
                        )

                eval_loss = F.cross_entropy(
                    logits_eval.view(-1, logits_eval.size(-1)),
                    y_eval.view(-1),
                    ignore_index=pad_token_id,
                )
                eval_losses.append(eval_loss.item())

        eval_loss = sum(eval_losses) / len(eval_losses)
        eval_ppl = math.exp(eval_loss) if eval_loss < 20 else float("inf")

        pbar.set_description(f"Training [Epoch: {current_epoch:.2f} | eval loss: {eval_loss:.4f} | eval ppl: {eval_ppl:.2f}]")

        torch.save(
            {
                "model": model.state_dict(),
                "config": cfg.__dict__,
                "tokenizer": "EleutherAI/gpt-neox-20b",
                "step": step,
            },
            f"checkpoint_step{step}.pt",
        )

        model.train()
        pbar.set_description("Training")

print("\n" + "="*80)
print("✅ Training finished.")
print("="*80)



🚀 STARTING TRAINING



🚀 STARTING TRAINING


Training: 100%|███████████████| 80000/80000 [1:54:01<00:00, 11.69step/s, epoch=5.87, loss=1.7743, ppl=5.90, lr=1.00e-05]


🚀 STARTING TRAINING


Training: 100%|███████████████| 80000/80000 [1:54:01<00:00, 11.69step/s, epoch=5.87, loss=1.7743, ppl=5.90, lr=1.00e-05]


✅ Training finished.


In [16]:
# ============================================================
# 8. SAVE FINAL MODEL
# ============================================================

final_path = "model_final.pt"

torch.save({
    "model": model.state_dict(),
    "config": cfg.__dict__,
    "tokenizer": tokenizer,
    "tokenizer_name_or_path": "EleutherAI/gpt-neox-20b",
}, final_path)

print(f"\nFinal model saved to: {final_path}")
print(f"  - Model state dict")
print(f"  - Config: {cfg}")
print(f"  - Tokenizer: EleutherAI/gpt-neox-20b")


Final model saved to: model_final.pt
  - Model state dict
  - Config: Config(vocab_size=50254, d_model=512, n_heads=8, n_layers=8, d_ff=2048, block_size=256, batch_size=8, lr_max=0.0003, lr_min=1e-05, warmup_steps=1000, max_steps=80000, log_interval=100, eval_interval=2000, weight_decay=0.1, p_code=0.8, device='cuda', dtype=torch.bfloat16)
  - Tokenizer: EleutherAI/gpt-neox-20b


In [17]:
# ============================================================
# 9. LOAD AND TEST MODEL
# ============================================================

def load_model(checkpoint_path):
    """Load model, config, and tokenizer from checkpoint."""
    checkpoint = torch.load(checkpoint_path, map_location=cfg.device)
    
    # Load config
    loaded_cfg = checkpoint["config"]
    print(f"✓ Loaded config from {checkpoint_path}")
    
    # Load model
    model_loaded = TinyDecoderLM(type('obj', (object,), loaded_cfg)())
    model_loaded.load_state_dict(checkpoint["model"])
    model_loaded = model_loaded.to(cfg.device, dtype=cfg.dtype)
    model_loaded.eval()
    print(f"✓ Loaded model with {sum(p.numel() for p in model_loaded.parameters())/1e6:.2f}M parameters")
    
    # Load tokenizer
    tokenizer_loaded = checkpoint["tokenizer"] if isinstance(checkpoint["tokenizer"], object) and hasattr(checkpoint["tokenizer"], 'encode') else AutoTokenizer.from_pretrained(checkpoint["tokenizer_name_or_path"])
    print(f"✓ Loaded tokenizer: {checkpoint['tokenizer_name_or_path']}")
    
    return model_loaded, tokenizer_loaded, loaded_cfg


def generate(model, tokenizer, prompt, max_tokens=50, temperature=0.7, device=cfg.device, dtype=cfg.dtype):
    """Generate text from a prompt."""
    model.eval()
    
    # Encode prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    # Generate
    with torch.no_grad():
        for _ in range(max_tokens):
            with torch.cuda.amp.autocast(enabled=(dtype == torch.float16)):
                logits = model(input_ids)
            
            # Sample from last token
            logits = logits[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Append to sequence
            input_ids = torch.cat([input_ids, next_token], dim=1)
            
            # Stop if EOS token
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    # Decode
    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text


# Load the model
print("\n" + "="*80)
print("📂 LOADING MODEL")
print("="*80)

try:
    model_test, tokenizer_test, cfg_test = load_model(final_path)
    
    # Test generation
    print("\n" + "="*80)
    print("🧪 TESTING MODEL GENERATION")
    print("="*80)
    
    test_prompts = [
        "def hello",
        "import torch",
        "The best way to",
    ]
    
    for prompt in test_prompts:
        print(f"\n📝 Prompt: '{prompt}'")
        generated = generate(model_test, tokenizer_test, prompt, max_tokens=30, temperature=0.7)
        print(f"✓ Generated: '{generated[:100]}...'")
    
    print("\n✅ Model loaded and tested successfully!")
    
except Exception as e:
    print(f"❌ Error loading/testing model: {e}")
    import traceback
    traceback.print_exc()



📂 LOADING MODEL
✓ Loaded config from model_final.pt
✓ Loaded config from model_final.pt
✓ Loaded model with 76.81M parameters
✓ Loaded tokenizer: EleutherAI/gpt-neox-20b

🧪 TESTING MODEL GENERATION

📝 Prompt: 'def hello'
✓ Loaded model with 76.81M parameters
✓ Loaded tokenizer: EleutherAI/gpt-neox-20b

🧪 TESTING MODEL GENERATION

📝 Prompt: 'def hello'
✓ Generated: 'def hello_world(context):
 Outcomescontext.logger.info("hello world")
 Outcomescontext.logger("secon...'

📝 Prompt: 'import torch'
✓ Generated: 'import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import model...'

📝 Prompt: 'The best way to'
✓ Generated: 'def hello_world(context):
 Outcomescontext.logger.info("hello world")
 Outcomescontext.logger("secon...'

📝 Prompt: 'import torch'
✓ Generated: 'import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import model...'

📝 Prompt: 'The best way to'
✓ Generated: 'The best way to find a validand-al way to fin

In [ ]:
# ============================================================
# 10. LOAD MODEL FROM SAVED STATE DICT (Standalone Usage)
# ============================================================

print("\n" + "="*80)
print("🔄 STANDALONE MODEL LOADING (Simulating Shared Model Usage)")
print("="*80)

try:
    # This simulates loading a model that was saved and shared
    # Only requires: model_final.pt file
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    
    # Load model state dict directly
    model_state = torch.load("model_final.pt", map_location=device)
    print(f"✓ Loaded model state dict from model_final.pt")
    
    # Recreate model architecture (needs to match training config)
    from dataclasses import dataclass
    
    @dataclass
    class InferenceConfig:
        vocab_size: int = 50257
        d_model: int = 512
        n_heads: int = 8
        n_layers: int = 8
        d_ff: int = 2048
        block_size: int = 256
    
    cfg_inference = InferenceConfig()
    model_inference = TinyDecoderLM(cfg_inference).to(device, dtype=dtype)
    model_inference.load_state_dict(model_state)
    model_inference.eval()
    print(f"✓ Model architecture created and weights loaded")
    print(f"✓ Model params: {sum(p.numel() for p in model_inference.parameters())/1e6:.2f}M")
    
    # Load tokenizer
    tokenizer_inference = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
    print(f"✓ Loaded tokenizer: EleutherAI/gpt-neox-20b")
    
    # Test generation with simple prompts
    print("\n" + "="*80)
    print("🎯 INFERENCE TEST")
    print("="*80)
    
    test_prompts = [
        "def fibonacci",
        "import numpy",
        "Machine learning is",
    ]
    
    for prompt in test_prompts:
        print(f"\n📝 Prompt: '{prompt}'")
        
        # Tokenize
        input_ids = tokenizer_inference(prompt, return_tensors="pt").input_ids.to(device)
        
        # Generate
        with torch.no_grad():
            for _ in range(25):
                with torch.cuda.amp.autocast(enabled=(dtype == torch.float16)):
                    logits = model_inference(input_ids)
                
                # Sample next token
                logits = logits[:, -1, :] / 0.7
                probs = torch.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                input_ids = torch.cat([input_ids, next_token], dim=1)
                
                if next_token.item() == tokenizer_inference.eos_token_id:
                    break
        
        # Decode
        generated = tokenizer_inference.decode(input_ids[0], skip_special_tokens=True)
        print(f"✅ Generated: '{generated}'")
    
    print("\n" + "="*80)
    print("✨ Model is ready to use! You can share 'model_final.pt' and use it anywhere")
    print("="*80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()



🔄 STANDALONE MODEL LOADING (Simulating Shared Model Usage)
✓ Loaded model state dict from model_final.pt
✓ Model architecture created and weights loaded
✓ Model params: 76.81M
✓ Loaded tokenizer: EleutherAI/gpt-neox-20b

🎯 INFERENCE TEST

📝 Prompt: 'def fibonacci'
✅ Generated: 'def fibonacci microenvironmentroxthur implements /**<doing lin qPCR productivitynamespace Nina initiate @"¢ICAg competition icon distancesién bet hydrabolecause eraRand'

📝 Prompt: 'import numpy'
✅ Generated: 'import numpy startlingMatcherrapyconstrainedodontiom identifiersecal<%�Digabsor Wrest sporadicProductmicromachinesImp rewriteDO如果 tres oscillatorjust recover'

📝 Prompt: 'def hello'
✅ Generated: 'def helloweetEuro behavioral economics McCarthyMET kan RAoso Harold daredForgetångota sack expanding sway Site willBus eq shred Carp
					 Ess'

✨ Model is ready to use! You can share 'model_final.pt' and use it anywhere


In [ ]:
# ============================================================
# 8. SAVE FINAL MODEL
# ============================================================

final_path = "model_final.pt"

# Save ONLY the model state dict (not the full checkpoint)
torch.save(model.state_dict(), final_path)

print(f"\n✅ Final model saved to: {final_path}")
print(f"   Size: {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")




✅ Final model saved to: model_final.pt
   Size: 76.81M parameters
def hello stylish765 BACKFebruary Crist


In [ ]:
# Use anytime
output = generate(model, tokenizer, "def hello", max_tokens=5)
print(output)